In [ ]:
import calendar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
from scipy.stats import linregress
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
import types

### Package requirements for reproducibility

In [ ]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

#### Define the initial dataset you'll be working on

In [ ]:
SatelliteJuly = pd.read_excel('Data.xlsx',sheet_name='July_sat').drop([r for r in range(12,24)])
SatelliteJune = pd.read_excel('Data.xlsx',sheet_name='June_sat').drop([r for r in range(12,24)])
SatelliteMay = pd.read_excel('Data.xlsx',sheet_name='May_sat').drop([r for r in range(12,24)])
SatelliteApril = pd.read_excel('Data.xlsx',sheet_name='April_satellite').drop([r for r in range(12,24)])
Seed = pd.read_excel('Data.xlsx',sheet_name='Chem_comp_wheat',usecols=[1,2,3,4,5,6,7,8]).drop([r for r in range(12,24)])
Dough = pd.read_excel('Data.xlsx',sheet_name='Dough',usecols=['W','P/L']).drop([r for r in range(12,24)])
Bread = pd.read_excel('Data.xlsx',sheet_name='Bread',usecols=[2,3,4,5,6,9]).drop([r for r in range(12,24)])
Seed2 = Seed.copy()
Seed2[['Prot sol acq','Prot nacl','Prot etoh', 'Prot ac ac']] = \
Seed[['Prot sol acq','Prot nacl','Prot etoh', 'Prot ac ac']].apply(lambda x: Seed.T.iloc[3].T * x/100)

### Print out the features of the fields we are running our inquiry on

In [ ]:
ta=pd.read_excel('Data.xlsx',sheet_name='Lots')
ta.head()

In [ ]:
SD = [SatelliteApril,SatelliteMay,SatelliteJune,SatelliteJuly]

SatelliteData = pd.concat(SD,axis=1,sort=False)

### Let's build a first regression between reflectance data and the features we are examining

In [ ]:
names = ['Seed','Dough','Bread']
db = [Seed,Dough,Bread]
Fc = [(names[idb],c) for idb,d in enumerate(db) for c in d ]
Features = pd.concat([Seed,Dough,Bread],axis=1)
Features.columns = pd.MultiIndex.from_tuples(Fc,names=('Stage','Feature'))

In [ ]:
scaler = StandardScaler()

PCA_v = []

PCA_f = []  

PCA_l = []



for sd in SD:

    sd=sd.dropna(axis='columns')

    scaler.fit(sd)
    scaledSatellite = pd.DataFrame(scaler.transform(sd))

    pca = PCA(.995)
    pca.fit(scaledSatellite)
    PCA_Satellite = pd.DataFrame(pca.transform(scaledSatellite))
    PCA_Satellite = PCA_Satellite.add_suffix(str(sd.columns[0][-7:])) 
    PCA_l.append(PCA_Satellite)

    PCA_stat = pd.DataFrame(pca.components_)
    PCA_stat['Explained_variance'] = pca.explained_variance_ratio_
    PCA_stat['month'] = calendar.month_abbr[int(sd.columns[0][-1])]
    PCA_f.append(PCA_stat)

PCA_df = pd.concat(PCA_l,axis=1,sort=False)

bl1 = []
bli = []
for ib in PCA_df:
    bl = []
    bli.append((calendar.month_abbr[int(ib[-1])],'PCA'+str(ib[0])))
    for c in Features:
        bl.append(linregress(PCA_df[ib],Features[c])[2]**2)
    bl1.append(bl)
Regression_PCA = pd.concat([pd.DataFrame(b1) for b1 in bl1],axis=1).T
Regression_PCA.index = pd.MultiIndex.from_tuples(bli,names=('Month','Index'))
Regression_PCA.columns = Features.columns
PCA_v.append(Regression_PCA)
    
PCA_variance = pd.concat(PCA_v).sort_index(level=0)

PCA_components = pd.concat(PCA_f).set_index(['month'])
PCA_components = PCA_components.sort_index(level=0)

In [ ]:
bl1 = []
bli = []
for ib in SatelliteData:
    bl = []
    bli.append((calendar.month_abbr[int(ib[-1])],ib[:4]))
    for c in Features:
        bl.append(linregress(SatelliteData[ib],Features[c])[2]**2)
    bl1.append(bl)
Regression_df = pd.concat([pd.DataFrame(b1) for b1 in bl1],axis=1).T
Regression_df.index = pd.MultiIndex.from_tuples(bli,names=('Month','Index'))
Regression_df.columns = Features.columns

### Query to see what is the highest regression figure

In [ ]:
Regression_month = Regression_PCA.groupby(level=[0])
Regression_index = Regression_PCA.groupby(level=[1])
Regression_feature = Regression_PCA.T.groupby(level=[0])

In [ ]:
Regressions = [Regression_month,Regression_index,Regression_feature]
for rs in Regressions:
    for idx, rf in rs:
        print(f'Feature: {idx}')
        print('-' * 16)
        print(rf[rf>0.1].count(), end='\n\n')
        print(rf[rf>0.1].max(), end='\n\n')

In [ ]:
Regressions = [Regression_month,Regression_index,Regression_feature]
for rs in Regressions:
    for idx, rf in rs:
        print(f'Feature: {idx}')
        print('-' * 16)
        print(rf[rf>0.3].count(), end='\n\n')
        print(rf[rf>0.3].max(), end='\n\n')